In [1]:
import numpy as np
import tensorflow as tf
import keras
from keras.models import load_model
import os
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.cluster import DBSCAN

from keras.optimizers import Adam

from full_view_model import fcn_model
from util_func import *
from full_view_train import *
from test_on_udacity_data import *
from convert_to_full_view_panorama import *

from keras.utils.generic_utils import get_custom_objects
get_custom_objects().update({"my_loss": my_loss})


Using TensorFlow backend.


In [2]:
# load trained model
model = load_model('./saved_model/fv_model_for_car_June_30_63.h5')

[<tf.Tensor 'gradients/split_1_grad/concat:0' shape=(?, 16, 320, 8) dtype=float32>, None, None]


In [3]:
data_dir = './data/training_didi_data/car_train_edited/'
list_of_view = list_of_data(data_dir)
print('len(list_of_view): ', len(list_of_view))
#list_of_view = ['./data/training_didi_data/car_train_edited/suburu_leading_front_left/view/view_281.npy',
#                './data/training_didi_data/car_train_edited/cmax_following_long/view/view_6631.npy']

len(list_of_view):  17977


In [4]:
list_of_lidar = [file.replace('view','lidar') for file in list_of_view]
list_of_gtbox = [file.replace('car_train_edited', 'car_train_gt_box_edited').replace('view','gt_boxes3d') 
                 for file in list_of_view]
print(list_of_lidar[0])
print(list_of_gtbox[0])

./data/training_didi_data/car_train_edited/suburu_driving_towards_it/lidar/lidar_196.npy
./data/training_didi_data/car_train_gt_box_edited/suburu_driving_towards_it/gt_boxes3d/gt_boxes3d_196.npy


### Predict and clustering predicted box

In [5]:
def one_box_clustering(boxes, eps = 1, min_samples = 1):
    # Extract the center from predicted boxes
    box_centers = np.mean(boxes[:,[0,2],:2], axis = 1)
    # Do clustering
    db = DBSCAN(eps=eps, min_samples=min_samples).fit(box_centers)
    labels = db.labels_
       
    n_clusters = len(set(labels))
    n_points = [np.sum([labels == i]) for i in range(n_clusters) ]
    max_point_cluster = np.argmax(n_points)
    
    index = (labels == max_point_cluster)
    box = np.mean(boxes[index],axis = 0)
    return np.expand_dims(box, 0)


def multi_box_clustering(boxes, eps = 1, min_samples = 1):
    # Extract the center from predicted boxes
    box_centers = np.mean(boxes[:,[0,2],:2], axis = 1)
    # Do clustering
    db = DBSCAN(eps=eps, min_samples=min_samples).fit(box_centers)
    labels = db.labels_
       
    n_clusters = len(set(labels))
    mul_clusters = np.array([np.mean(boxes[labels == i],axis = 0) for i in range(n_clusters)])
    
    return mul_clusters
    
    

In [6]:
def distance(p,q):
    return np.sqrt(np.sum(np.square(p-q)))

def length(v):
    return distance(v,0)

def fit_box(lidar, nb_d = 128):
    
    lidar_2d = lidar[:,:2]
    angle = np.pi/(nb_d*2)
    
    rotated_lidar = np.array([rotation(angle*i, lidar_2d) for i in range(nb_d)])
    max_lidars = np.max(rotated_lidar, axis = 1)
    min_lidars = np.min(rotated_lidar, axis = 1)
    range_lidars = max_lidars - min_lidars
    areas = range_lidars[:,0]*range_lidars[:,1]
    
    arg_min = np.argmin(areas)
    
    rp0, rp2 = min_lidars[arg_min], max_lidars[arg_min]
    rp1, rp3 = np.array([rp2[0], rp0[1]]), np.array([rp0[0], rp2[1]])
    
    box_2d = np.array([rotation(-angle*arg_min, p) for p in [rp0,rp1,rp2,rp3]])
    # box = -1.5*np.ones((1,8,3))
    # box[0,:4,:2] = box_2d
    # box[0,4:,:2] = box_2d
    # box[0,4:,2] = np.max(lidar[:,2])
    return box_2d#, box

def move_box(box, side):
    '''
    box: 2d box of shape (4,2)
    side: array of shape (2,2)
    '''

    #print(box)
    #print(side)
    v_box = box[1] - box[0]
    v_side = side[1] - side[0]
    angle_offset = np.arctan2(v_side[1], v_side[0]) - np.arctan2(v_box[1], v_box[0])
    #print(np.arctan2(v_side[1], v_side[0]), np.arctan2(v_box[1], v_box[0]))
    #print(angle_offset)
    
    rot_box = np.array([rotation(-angle_offset, box[i] - box[0]) + box[0] for i in range(4)])
    #print('rot_box')
    #print(rot_box)
    
    pos_offset = side[0] - box[0]
    correct_box = rot_box + np.expand_dims(pos_offset, axis = 0)
     
    return correct_box

def correct_box(predbox, fitbox):
    '''
    box, fitbox: 2d box of shape (4,2)
    Move box to the right position based on position of fitbox 
    '''
    pred_sides = np.array([distance(predbox[0], predbox[1]), distance(predbox[1], predbox[2])])
    min_pred_side = np.min(pred_sides)
    min_pred_ind = np.argmin(pred_sides)
    
    fit_distances = np.array([length(fitbox[i]) for i in range(4)])  
    min_fit_ind = np.argmin(fit_distances)
    next_fit_ind = (min_fit_ind + 1)%4
    prev_fit_ind = (min_fit_ind + 3)%4
    
    next_fit_side = distance(fitbox[min_fit_ind],fitbox[next_fit_ind])
    prev_fit_side = distance(fitbox[min_fit_ind],fitbox[prev_fit_ind])
    
    nearest_fit_point = fitbox[min_fit_ind]
    
    diff_next_side = abs(next_fit_side - min_pred_side)
    diff_prev_side = abs(prev_fit_side - min_pred_side)
    if diff_next_side < diff_prev_side:
        side = fitbox[[min_fit_ind, next_fit_ind],:]
    else:
        side = fitbox[[prev_fit_ind, min_fit_ind],:]
    
    indices = [(min_pred_ind + i)%4 for i in range(4)]
    box = np.array([predbox[i] for i in indices])
    box = move_box(box, side)
    box_distances = np.array([length(box[i]) for i in range(4)])
    min_ind = np.argmin(box_distances)
    if min_ind == min_fit_ind:
        return box
    else:
        indices = [(min_pred_ind + i + 2)%4 for i in range(4)]
        box = np.array([predbox[i] for i in indices])
        box = move_box(box, side)
        return box
    
def correct_box_3d(predbox, fitbox, min_z = -1.5):
    '''
    box: 3d box of shape (8,3)
    fitbox: 2d box of shape (4,2)
    Move box to the right position based on position of fitbox 
    '''
    box2d = correct_box(predbox[:4,:2], fitbox)
    box = min_z*np.ones_like(predbox)
    box[:4,:2] = box2d
    box[4:,:2] = box2d
    box[4:,2] = abs(predbox[4,2] - predbox[0,2])+min_z
    return box

In [21]:
def correct_predicted_box(clusters, labels, boxes):
    list_clusters = list(set(labels))
    nb_clusters = len(list_clusters)
    if nb_clusters == 0:
        return np.array([])
#     if nb_clusters == 1:
#         assert len(boxes) == 1, 'nb_clusters == 1 but len(boxes) = {0}'.format(len(boxes))
#         fitbox = fit_box(clusters)
#         correctbox = correct_box_3d(boxes[0], fitbox)
#         correctbox = np.expand_dims(correctbox, axis = 0)
#         return correctbox
    else:
        correctboxes = np.zeros((len(boxes), 8, 3))
        list_of_cluster = [clusters[labels == list_clusters[i]] for i in range(nb_clusters)]
        center_clusters = [np.mean(list_of_cluster[i][:,:2], axis = 0) for i in range(nb_clusters)]
        for b in range(len(boxes)):
            box = boxes[b]
            center_box = np.mean(box[:4,:2], axis = 0)
            distances = [distance(center_box, center_clusters[i]) for i in range(nb_clusters)]
            ind = np.argmin(distances)
            nearest_cluster = list_of_cluster[ind]
            
            fitbox = fit_box(nearest_cluster)
            correctbox = correct_box_3d(box, fitbox)
            
            correctboxes[b] = correctbox
        return correctboxes
            
            
            
    

In [8]:
def predict(model,lidar, cluster = True, seg_thres=0.5, multi_box = True):
    
    test_view, clusters, labels =  fv_cylindrical_projection_for_test(lidar)
    
    view = test_view[:,:,[5,2]].reshape(1,16,320,2)
    
    list_boxes = []

    test_view_reshape = test_view.reshape(-1,6)
    pred = model.predict(view)
    pred = pred[0].reshape(-1,8)
    
    thres_pred = pred[pred[:,0] > seg_thres]
    thres_view = test_view_reshape[pred[:,0] > seg_thres]
    
    
    num_boxes = len(thres_pred)
    if num_boxes == 0:
        return np.array([]), np.array([])
    boxes = np.zeros((num_boxes,8,3))
    
    for i in range(num_boxes):
        boxes[i,0] = thres_view[i,:3] - rotation(thres_view[i,3],thres_pred[i,1:4])
        boxes[i,6] = thres_view[i,:3] - rotation(thres_view[i,3],thres_pred[i,4:7])

        boxes[i,2,:2] = boxes[i,6,:2]
        boxes[i,2,2] = boxes[i,0,2]

        phi = thres_pred[i,-1]

        z = boxes[i,2] - boxes[i,0]
        boxes[i,1,0] = (np.cos(phi)*z[0] + np.sin(phi)*z[1])*np.cos(phi) + boxes[i,0,0]
        boxes[i,1,1] = (-np.sin(phi)*z[0] + np.cos(phi)*z[1])*np.cos(phi) + boxes[i,0,1]
        boxes[i,1,2] = boxes[i,0,2]

        boxes[i,3] = boxes[i,0] + boxes[i,2] - boxes[i,1]
        boxes[i,4] = boxes[i,0] + boxes[i,6] - boxes[i,2]
        boxes[i,5] = boxes[i,1] + boxes[i,4] - boxes[i,0]
        boxes[i,7] = boxes[i,4] + boxes[i,6] - boxes[i,5]
        
    list_boxes.append(boxes)

    boxes = np.concatenate(list_boxes, axis = 0)
    if not cluster:
        return boxes
    
    elif multi_box:
        mul_clusters = multi_box_clustering(boxes)
        return boxes, mul_clusters
    else:
        one_cluster = one_box_clustering(boxes)
        return boxes, one_cluster
    

In [9]:
def predict_and_correct(model,lidar, cluster = True, seg_thres=0.5, multi_box = True):
    
    test_view, clusters, labels =  fv_cylindrical_projection_for_test(lidar)
    
    view = test_view[:,:,[5,2]].reshape(1,16,320,2)
    
    list_boxes = []

    test_view_reshape = test_view.reshape(-1,6)
    pred = model.predict(view)
    pred = pred[0].reshape(-1,8)
    
    thres_pred = pred[pred[:,0] > seg_thres]
    thres_view = test_view_reshape[pred[:,0] > seg_thres]
    
    
    num_boxes = len(thres_pred)
    if num_boxes == 0:
        return np.array([]), np.array([])
    boxes = np.zeros((num_boxes,8,3))
    
    for i in range(num_boxes):
        boxes[i,0] = thres_view[i,:3] - rotation(thres_view[i,3],thres_pred[i,1:4])
        boxes[i,6] = thres_view[i,:3] - rotation(thres_view[i,3],thres_pred[i,4:7])

        boxes[i,2,:2] = boxes[i,6,:2]
        boxes[i,2,2] = boxes[i,0,2]

        phi = thres_pred[i,-1]

        z = boxes[i,2] - boxes[i,0]
        boxes[i,1,0] = (np.cos(phi)*z[0] + np.sin(phi)*z[1])*np.cos(phi) + boxes[i,0,0]
        boxes[i,1,1] = (-np.sin(phi)*z[0] + np.cos(phi)*z[1])*np.cos(phi) + boxes[i,0,1]
        boxes[i,1,2] = boxes[i,0,2]

        boxes[i,3] = boxes[i,0] + boxes[i,2] - boxes[i,1]
        boxes[i,4] = boxes[i,0] + boxes[i,6] - boxes[i,2]
        boxes[i,5] = boxes[i,1] + boxes[i,4] - boxes[i,0]
        boxes[i,7] = boxes[i,4] + boxes[i,6] - boxes[i,5]
        
    list_boxes.append(boxes)

    boxes = np.concatenate(list_boxes, axis = 0)
    if not cluster:
        return boxes
    
    elif multi_box:
        mul_boxes = multi_box_clustering(boxes)
        mul_boxes = correct_predicted_box(clusters, labels, mul_boxes)
        return boxes, mul_boxes
    else:
        one_box = one_box_clustering(boxes)
        one_box = correct_predicted_box(clusters, labels, one_box)
        return boxes, one_box
    

In [10]:
# randomly choose a frame 
i = 400
#i = np.random.randint(len(list_of_lidar))
lidar = np.load(list_of_lidar[i])
gtbox = np.load(list_of_gtbox[i])

boxes, box_cluster = predict(model, lidar)

In [11]:
print(boxes.shape), print(box_cluster.shape)
n_boxes = len(boxes)
print('Number of predited boxes: {}'.format(n_boxes))
print('Number of boxes after clustering: {}'.format(len(box_cluster)))
print('Randomly display 10 boxes then display clustered boxes')
choosen_boxes = np.random.randint(n_boxes, size = 10)
viz_mayavi_with_labels(lidar, boxes[choosen_boxes])
viz_mayavi_with_labels(lidar, box_cluster)

(4, 8, 3)
(1, 8, 3)
Number of predited boxes: 4
Number of boxes after clustering: 1
Randomly display 10 boxes then display clustered boxes


In [12]:
boxes, box_cluster = predict_and_correct(model, lidar)

In [13]:
viz_mayavi_with_labels(lidar, box_cluster)

### predict on test data

In [14]:
#test_dir = './data/test_cars/'
test_lidar = np.load('./data/training_didi_data/car_train_edited/suburu_driving_towards_it/lidar/lidar_140.npy')
#test_lidar = np.load('./data/test_cars/ford01/lidar_0.npy')
boxes, box_cluster = predict(model, test_lidar)
n_boxes = len(boxes)
print('Number of predited boxes: {}'.format(n_boxes))
print('Number of boxes after clustering: {}'.format(len(box_cluster)))
print('Randomly display 10 boxes then display clustered boxes')
choosen_boxes = np.random.randint(n_boxes, size = 10)
viz_mayavi_with_labels(test_lidar, boxes[choosen_boxes])
viz_mayavi_with_labels(test_lidar, box_cluster)


Number of predited boxes: 5
Number of boxes after clustering: 1
Randomly display 10 boxes then display clustered boxes


In [15]:
boxes, new_box_cluster = predict_and_correct(model, test_lidar)
n_boxes = len(boxes)
print('Number of predited boxes: {}'.format(n_boxes))
print('Number of boxes after clustering: {}'.format(len(box_cluster)))
print('Randomly display 10 boxes then display clustered boxes')
choosen_boxes = np.random.randint(n_boxes, size = 10)
viz_mayavi_with_labels(test_lidar, boxes[choosen_boxes])
viz_mayavi_with_labels(test_lidar, new_box_cluster)

Number of predited boxes: 5
Number of boxes after clustering: 1
Randomly display 10 boxes then display clustered boxes


In [16]:
def predict_bags(model, test_dir, box_test_dir):
    '''
    This function predict all the test file and save predicted boxes in box_test_dir
    '''
    if not os.path.exists(box_test_dir):
        os.mkdir(box_test_dir)
    
    bags = os.listdir(test_dir)
    for bag in bags:
        box_dir = os.path.join(box_test_dir, bag)
        if not os.path.exists(box_dir):
            os.mkdir(box_dir)
        lidar_dir = os.path.join(test_dir, bag)
        for f in os.listdir(lidar_dir):
            lidar = np.load(os.path.join(lidar_dir,f))
            #print(lidar_dir, f)
            boxes, box_cluster = predict_and_correct(model, lidar)
            #print('-----------------')
            #f_box = f.replace('lidar', 'boxes')
            f_cluster = f.replace('lidar', 'boxes')
            #box_file = os.path.join(box_dir, f_box)
            #np.save(box_file, boxes)
            cluster_file = os.path.join(box_dir, f_cluster)
            np.save(cluster_file, box_cluster)
    

In [ ]:
test_dir = './data/test_cars/'
box_test_dir = './data/new_predicted_test_cars/'
predict_bags(model, test_dir, box_test_dir)

./data/test_cars/ford04 lidar_156.npy
-----------------
./data/test_cars/ford04 lidar_68.npy
-----------------
./data/test_cars/ford04 lidar_28.npy
-----------------
./data/test_cars/ford04 lidar_76.npy
-----------------
./data/test_cars/ford04 lidar_66.npy
-----------------
./data/test_cars/ford04 lidar_108.npy
-----------------
./data/test_cars/ford04 lidar_94.npy
-----------------
./data/test_cars/ford04 lidar_59.npy
-----------------
./data/test_cars/ford04 lidar_9.npy
-----------------
./data/test_cars/ford04 lidar_6.npy
-----------------
./data/test_cars/ford04 lidar_121.npy
-----------------
./data/test_cars/ford04 lidar_72.npy
-----------------
./data/test_cars/ford04 lidar_22.npy
-----------------
./data/test_cars/ford04 lidar_152.npy
-----------------
./data/test_cars/ford04 lidar_65.npy
-----------------
./data/test_cars/ford04 lidar_51.npy
-----------------
./data/test_cars/ford04 lidar_83.npy
-----------------
./data/test_cars/ford04 lidar_150.npy
-----------------
./data/

In [ ]:
lidar = np.load('./data/test_cars/ford03/lidar_100.npy')
pred_boxes = np.load('./data/box_test_cars/ford03/boxes_100.npy')
pred_box = np.load('./data/box_test_cars/ford03/clustered_box_100.npy')
print(pred_boxes.shape, pred_box.shape)

In [ ]:
viz_mayavi_with_labels(lidar, pred_boxes)
viz_mayavi_with_labels(lidar, pred_box)